In [1]:
USE_VIT = True

if USE_VIT:
    import dqn_vit as dqn
else:
    import dqn

In [2]:
import json
import os
import time
from distutils.dir_util import copy_tree
import gym_unrealcv
import gym
from constants import *
from example.utils import preprocessing, io_util
from gym import wrappers

In [3]:
env = gym.make(ENV_NAME)

# env.rendering = SHOW
# must be discrete action
assert env.action_type == 'Discrete'

exec nohup /home/eee198/Documents/gym-unrealcv/gym_unrealcv/envs/UnrealEnv/RealisticRendering_RL_3.10/RealisticRendering/Binaries/Linux/RealisticRendering 
Running docker-free env, pid:17427
Please wait for a while to launch env......


INFO:__init__:192:Got connection confirm: b'connected to RealisticRendering'


127.0.0.1


/home/eee198/anaconda3/envs/vit-rl/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Environment '<class 'gym_unrealcv.envs.unrealcv_search_base.UnrealCvSearch_base'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
ACTION_SIZE = env.action_space.n
#ACTION_LIST = env.discrete_actions
INPUT_CHANNELS = env.observation_space.shape[2]
OBS_HIGH = env.observation_space.high
OBS_LOW = env.observation_space.low
OBS_RANGE = OBS_HIGH - OBS_LOW

process_img = preprocessing.preprocessor(observation_space=env.observation_space, length=INPUT_CHANNELS,
                                         size=(INPUT_SIZE, INPUT_SIZE))

#init log file
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
if not os.path.exists(PARAM_DIR):
    os.makedirs(PARAM_DIR)

In [5]:
#load init param
CONTINUE=False

if not CONTINUE:
    explorationRate = INITIAL_EPSILON
    current_epoch = 0
    stepCounter = 0
    loadsim_seconds = 0
    Agent = dqn.DeepQ(ACTION_SIZE, MEMORY_SIZE, GAMMA, LEARNING_RATE,
                      INPUT_SIZE, INPUT_SIZE,INPUT_CHANNELS,USE_TARGET_NETWORK)
    env = wrappers.Monitor(env, MONITOR_DIR + 'tmp', write_upon_reset=True,force=True)

    io_util.create_csv_header(TRA_DIR)
else:
    #load init param

    #Load weights, monitor info and parameter info.

    explorationRate = 0.1
    current_epoch = 0
    stepCounter = 0
    loadsim_seconds = 0
    Agent = dqn.DeepQ(
        ACTION_SIZE,
        MEMORY_SIZE,
        GAMMA,
        LEARNING_RATE,
        INPUT_SIZE,
        INPUT_SIZE,
        INPUT_CHANNELS,
        USE_TARGET_NETWORK
    )
    Agent.loadWeights('/home/eee198/Documents/gym-unrealcv/example/dqn_torch/saved_weights/dqn_ep5000.pt')
    env = wrappers.Monitor(env, MONITOR_DIR + 'tmp', write_upon_reset=True,force=True)

    io_util.create_csv_header(TRA_DIR)

In [ ]:
#main loop
try:
    start_time = time.time()
    for epoch in range(current_epoch, MAX_EPOCHS):
        obs = env.reset()
        #observation = io_util.preprocess_img((obs-OBS_LOW)/OBS_RANGE)
        observation = process_img.process_gray(obs, reset=True)
        cumulated_reward = 0
        if (epoch % TEST_INTERVAL_EPOCHS != 0 or stepCounter < LEARN_START_STEP) and TRAIN is True :  # explore
            EXPLORE = True
        else:
            EXPLORE = False
            print("Evaluate Model")
        for t in range(1000):

            start_req = time.time()

            if EXPLORE is True: #explore
                action = Agent.feedforward(observation, explorationRate)
                obs_new, reward, done, info = env.step(action)
                #newObservation = io_util.preprocess_img((obs_new-OBS_LOW)/OBS_RANGE)
                newObservation = process_img.process_gray(obs_new)
                stepCounter += 1
                Agent.addMemory(observation, action, reward, newObservation, done)
                observation = newObservation
                if stepCounter == LEARN_START_STEP:
                    print("Starting learning")

                if Agent.getMemorySize() >= LEARN_START_STEP:
                    Agent.learnOnMiniBatch(BATCH_SIZE)

                    if explorationRate > FINAL_EPSILON and stepCounter > LEARN_START_STEP:
                        explorationRate -= (INITIAL_EPSILON - FINAL_EPSILON) / MAX_EXPLORE_STEPS
                    #elif stepCounter%(MAX_EXPLORE_STEPS * 1.5) == 0 :
                        #explorationRate = 0.99
                        #print 'Reset Exploration Rate'

            #test
            else:
                action = Agent.feedforward(observation,0)
                obs_new, reward, done, info = env.step(action)
                newObservation = process_img.process_gray(obs_new)
                #newObservation = io_util.preprocess_img((obs_new-OBS_LOW)/OBS_RANGE)
                observation = newObservation

            if MAP:
                io_util.live_plot(info)

            #io_util.save_trajectory(info,TRA_DIR,epoch)

            cumulated_reward += reward
            if done:
                m, s = divmod(int(time.time() - start_time + loadsim_seconds), 60)
                h, m = divmod(m, 60)

                print ("EP " + str(epoch) +" Csteps= " + str(stepCounter) + " - {} steps".format(t + 1) + " - CReward: " + str(
                    round(cumulated_reward, 2)) + "  Eps=" + str(round(explorationRate, 2)) + "  Time: %d:%02d:%02d" % (h, m, s) )
                    # SAVE SIMULATION DATA
                if epoch % SAVE_INTERVAL_EPOCHS == 0 and TRAIN is True:
                    # save model weights and monitoring data
                    print('Save model')
                    Agent.saveModel(MODEL_DIR + '/dqn_ep' + str(epoch) + '.pt')

                    #backup monitor file
                    copy_tree(MONITOR_DIR+ 'tmp', MONITOR_DIR + str(epoch))

                    parameter_keys = ['explorationRate', 'current_epoch','stepCounter', 'FINAL_EPSILON','loadsim_seconds']
                    parameter_values = [explorationRate, epoch, stepCounter,FINAL_EPSILON, int(time.time() - start_time + loadsim_seconds)]
                    parameter_dictionary = dict(zip(parameter_keys, parameter_values))
                    with open(PARAM_DIR + '/dqn_ep' + str(epoch) + '.json','w') as outfile:
                        json.dump(parameter_dictionary, outfile)



                break



except KeyboardInterrupt:
    print("Shutting down")
    env.close()

EP 0 Csteps= 1 - 1 steps - CReward: -1  Eps=1  Time: 0:00:00
Save model
EP 1 Csteps= 5 - 4 steps - CReward: -0.85  Eps=1  Time: 0:00:01
EP 2 Csteps= 10 - 5 steps - CReward: 9.07  Eps=1  Time: 0:00:01
EP 3 Csteps= 13 - 3 steps - CReward: -1.06  Eps=1  Time: 0:00:02
EP 4 Csteps= 24 - 11 steps - CReward: -1.88  Eps=1  Time: 0:00:03
EP 5 Csteps= 44 - 20 steps - CReward: -0.36  Eps=1  Time: 0:00:06
EP 6 Csteps= 50 - 6 steps - CReward: -2.09  Eps=1  Time: 0:00:07
EP 7 Csteps= 61 - 11 steps - CReward: -2.4  Eps=1  Time: 0:00:08
EP 8 Csteps= 68 - 7 steps - CReward: -0.88  Eps=1  Time: 0:00:09
EP 9 Csteps= 79 - 11 steps - CReward: -4.18  Eps=1  Time: 0:00:10
EP 10 Csteps= 84 - 5 steps - CReward: -0.95  Eps=1  Time: 0:00:11
EP 11 Csteps= 92 - 8 steps - CReward: -2.73  Eps=1  Time: 0:00:12
EP 12 Csteps= 98 - 6 steps - CReward: -1.31  Eps=1  Time: 0:00:13
EP 13 Csteps= 107 - 9 steps - CReward: -3.32  Eps=1  Time: 0:00:14
EP 14 Csteps= 115 - 8 steps - CReward: -0.88  Eps=1  Time: 0:00:16
EP 15 Cste

EP 123 Csteps= 809 - 10 steps - CReward: -0.2  Eps=1  Time: 0:01:52
EP 124 Csteps= 816 - 7 steps - CReward: -0.52  Eps=1  Time: 0:01:53
EP 125 Csteps= 825 - 9 steps - CReward: -4.5  Eps=1  Time: 0:01:54
EP 126 Csteps= 831 - 6 steps - CReward: -3.08  Eps=1  Time: 0:01:55
EP 127 Csteps= 839 - 8 steps - CReward: -1.14  Eps=1  Time: 0:01:56
EP 128 Csteps= 842 - 3 steps - CReward: 1.93  Eps=1  Time: 0:01:57
EP 129 Csteps= 848 - 6 steps - CReward: -2.47  Eps=1  Time: 0:01:58
EP 130 Csteps= 856 - 8 steps - CReward: -0.42  Eps=1  Time: 0:01:59
EP 131 Csteps= 862 - 6 steps - CReward: -1.74  Eps=1  Time: 0:02:00
EP 132 Csteps= 864 - 2 steps - CReward: -1.06  Eps=1  Time: 0:02:00
EP 133 Csteps= 869 - 5 steps - CReward: -0.92  Eps=1  Time: 0:02:01
EP 134 Csteps= 871 - 2 steps - CReward: -1.02  Eps=1  Time: 0:02:01
EP 135 Csteps= 878 - 7 steps - CReward: -0.72  Eps=1  Time: 0:02:02
EP 136 Csteps= 885 - 7 steps - CReward: -2.15  Eps=1  Time: 0:02:03
EP 137 Csteps= 886 - 1 steps - CReward: 2.14  Eps=

EP 243 Csteps= 1603 - 15 steps - CReward: -2.7  Eps=1  Time: 0:03:44
EP 244 Csteps= 1609 - 6 steps - CReward: -0.28  Eps=1  Time: 0:03:44
EP 245 Csteps= 1612 - 3 steps - CReward: -1.19  Eps=1  Time: 0:03:45
EP 246 Csteps= 1617 - 5 steps - CReward: -1.13  Eps=1  Time: 0:03:45
EP 247 Csteps= 1620 - 3 steps - CReward: 1.55  Eps=1  Time: 0:03:46
EP 248 Csteps= 1628 - 8 steps - CReward: 2.61  Eps=1  Time: 0:03:47
EP 249 Csteps= 1631 - 3 steps - CReward: 2.73  Eps=1  Time: 0:03:47
EP 250 Csteps= 1638 - 7 steps - CReward: -1.14  Eps=1  Time: 0:03:48
EP 251 Csteps= 1642 - 4 steps - CReward: -1.33  Eps=1  Time: 0:03:49
EP 252 Csteps= 1646 - 4 steps - CReward: -1.44  Eps=1  Time: 0:03:50
EP 253 Csteps= 1660 - 14 steps - CReward: -3.67  Eps=1  Time: 0:03:52
EP 254 Csteps= 1669 - 9 steps - CReward: 2.17  Eps=1  Time: 0:03:53
EP 255 Csteps= 1674 - 5 steps - CReward: -0.88  Eps=1  Time: 0:03:54
EP 256 Csteps= 1688 - 14 steps - CReward: -0.4  Eps=1  Time: 0:03:55
EP 257 Csteps= 1697 - 9 steps - CRewa

EP 363 Csteps= 2420 - 8 steps - CReward: -3.35  Eps=1  Time: 0:05:38
EP 364 Csteps= 2434 - 14 steps - CReward: -4.07  Eps=1  Time: 0:05:40
EP 365 Csteps= 2438 - 4 steps - CReward: -0.88  Eps=1  Time: 0:05:40
EP 366 Csteps= 2446 - 8 steps - CReward: -1.98  Eps=1  Time: 0:05:41
EP 367 Csteps= 2451 - 5 steps - CReward: -1.06  Eps=1  Time: 0:05:42
EP 368 Csteps= 2460 - 9 steps - CReward: -0.3  Eps=1  Time: 0:05:43
EP 369 Csteps= 2464 - 4 steps - CReward: -1.16  Eps=1  Time: 0:05:44
EP 370 Csteps= 2469 - 5 steps - CReward: 3.36  Eps=1  Time: 0:05:44
EP 371 Csteps= 2471 - 2 steps - CReward: -0.96  Eps=1  Time: 0:05:45
EP 372 Csteps= 2474 - 3 steps - CReward: -1.03  Eps=1  Time: 0:05:45
EP 373 Csteps= 2480 - 6 steps - CReward: 2.41  Eps=1  Time: 0:05:46
EP 374 Csteps= 2489 - 9 steps - CReward: -2.01  Eps=1  Time: 0:05:47
EP 375 Csteps= 2495 - 6 steps - CReward: -2.44  Eps=1  Time: 0:05:48
EP 376 Csteps= 2499 - 4 steps - CReward: 1.71  Eps=1  Time: 0:05:49
EP 377 Csteps= 2506 - 7 steps - CRewa

EP 483 Csteps= 3200 - 6 steps - CReward: 0.08  Eps=1  Time: 0:07:26
EP 484 Csteps= 3208 - 8 steps - CReward: -0.34  Eps=1  Time: 0:07:27
EP 485 Csteps= 3217 - 9 steps - CReward: 1.09  Eps=1  Time: 0:07:28
EP 486 Csteps= 3218 - 1 steps - CReward: 1.14  Eps=1  Time: 0:07:28
EP 487 Csteps= 3222 - 4 steps - CReward: -0.9  Eps=1  Time: 0:07:29
EP 488 Csteps= 3227 - 5 steps - CReward: 1.56  Eps=1  Time: 0:07:30
EP 489 Csteps= 3238 - 11 steps - CReward: -3.21  Eps=1  Time: 0:07:31
EP 490 Csteps= 3241 - 3 steps - CReward: -0.92  Eps=1  Time: 0:07:32
EP 491 Csteps= 3243 - 2 steps - CReward: 6.42  Eps=1  Time: 0:07:32
EP 492 Csteps= 3259 - 16 steps - CReward: 0.23  Eps=1  Time: 0:07:34
EP 493 Csteps= 3267 - 8 steps - CReward: 0.38  Eps=1  Time: 0:07:35
EP 494 Csteps= 3270 - 3 steps - CReward: -1.0  Eps=1  Time: 0:07:36
EP 495 Csteps= 3275 - 5 steps - CReward: 1.72  Eps=1  Time: 0:07:36
EP 496 Csteps= 3284 - 9 steps - CReward: -1.37  Eps=1  Time: 0:07:38
EP 497 Csteps= 3289 - 5 steps - CReward: -

...

EP 5717 Csteps= 49295 - 2 steps - CReward: 1.84  Eps=0.1  Time: 2:57:11
EP 5718 Csteps= 49297 - 2 steps - CReward: 2.01  Eps=0.1  Time: 2:57:11
EP 5719 Csteps= 49302 - 5 steps - CReward: 3.68  Eps=0.1  Time: 2:57:13
EP 5720 Csteps= 49324 - 22 steps - CReward: 11.33  Eps=0.1  Time: 2:57:17
EP 5721 Csteps= 49336 - 12 steps - CReward: 10.28  Eps=0.1  Time: 2:57:20
EP 5722 Csteps= 49346 - 10 steps - CReward: 3.12  Eps=0.1  Time: 2:57:22
EP 5723 Csteps= 49358 - 12 steps - CReward: 9.94  Eps=0.1  Time: 2:57:25
EP 5724 Csteps= 49370 - 12 steps - CReward: 8.52  Eps=0.1  Time: 2:57:29
EP 5725 Csteps= 49378 - 8 steps - CReward: 0.01  Eps=0.1  Time: 2:57:31
EP 5726 Csteps= 49383 - 5 steps - CReward: 9.12  Eps=0.1  Time: 2:57:32
EP 5727 Csteps= 49400 - 17 steps - CReward: -1.82  Eps=0.1  Time: 2:57:36
EP 5728 Csteps= 49411 - 11 steps - CReward: 10.91  Eps=0.1  Time: 2:57:38
EP 5729 Csteps= 49416 - 5 steps - CReward: 4.09  Eps=0.1  Time: 2:57:40
EP 5730 Csteps= 49431 - 15 steps - CReward: 11.67  Ep

EP 5830 Csteps= 50384 - 13 steps - CReward: 10.42  Eps=0.1  Time: 3:01:24
EP 5831 Csteps= 50393 - 9 steps - CReward: 6.32  Eps=0.1  Time: 3:01:27
EP 5832 Csteps= 50397 - 4 steps - CReward: -0.91  Eps=0.1  Time: 3:01:28
EP 5833 Csteps= 50412 - 15 steps - CReward: 11.58  Eps=0.1  Time: 3:01:31
EP 5834 Csteps= 50421 - 9 steps - CReward: -0.06  Eps=0.1  Time: 3:01:33
EP 5835 Csteps= 50434 - 13 steps - CReward: 10.74  Eps=0.1  Time: 3:01:36
EP 5836 Csteps= 50449 - 15 steps - CReward: -0.45  Eps=0.1  Time: 3:01:39
EP 5837 Csteps= 50458 - 9 steps - CReward: 10.49  Eps=0.1  Time: 3:01:41
EP 5838 Csteps= 50468 - 10 steps - CReward: 10.27  Eps=0.1  Time: 3:01:44
EP 5839 Csteps= 50487 - 19 steps - CReward: 9.07  Eps=0.1  Time: 3:01:48
EP 5840 Csteps= 50500 - 13 steps - CReward: 10.2  Eps=0.1  Time: 3:01:51
EP 5841 Csteps= 50504 - 4 steps - CReward: -1.12  Eps=0.1  Time: 3:01:52
EP 5842 Csteps= 50510 - 6 steps - CReward: 3.16  Eps=0.1  Time: 3:01:53
EP 5843 Csteps= 50516 - 6 steps - CReward: 9.67 

EP 5942 Csteps= 51615 - 15 steps - CReward: 9.68  Eps=0.1  Time: 3:06:10
EP 5943 Csteps= 51623 - 8 steps - CReward: 10.87  Eps=0.1  Time: 3:06:11
EP 5944 Csteps= 51631 - 8 steps - CReward: 2.31  Eps=0.1  Time: 3:06:13
EP 5945 Csteps= 51646 - 15 steps - CReward: 7.0  Eps=0.1  Time: 3:06:17
EP 5946 Csteps= 51648 - 2 steps - CReward: 3.82  Eps=0.1  Time: 3:06:17
EP 5947 Csteps= 51663 - 15 steps - CReward: 10.78  Eps=0.1  Time: 3:06:21
EP 5948 Csteps= 51671 - 8 steps - CReward: -2.13  Eps=0.1  Time: 3:06:23
EP 5949 Csteps= 51683 - 12 steps - CReward: 11.56  Eps=0.1  Time: 3:06:27
EP 5950 Csteps= 51693 - 10 steps - CReward: -0.03  Eps=0.1  Time: 3:06:29
EP 5951 Csteps= 51700 - 7 steps - CReward: 10.2  Eps=0.1  Time: 3:06:31
EP 5952 Csteps= 51707 - 7 steps - CReward: 10.72  Eps=0.1  Time: 3:06:32
EP 5953 Csteps= 51717 - 10 steps - CReward: 11.45  Eps=0.1  Time: 3:06:34
EP 5954 Csteps= 51728 - 11 steps - CReward: 9.12  Eps=0.1  Time: 3:06:37
EP 5955 Csteps= 51736 - 8 steps - CReward: 8.84  Ep

EP 6055 Csteps= 52752 - 13 steps - CReward: 9.69  Eps=0.1  Time: 3:10:32
EP 6056 Csteps= 52759 - 7 steps - CReward: 11.5  Eps=0.1  Time: 3:10:34
EP 6057 Csteps= 52767 - 8 steps - CReward: 1.06  Eps=0.1  Time: 3:10:36
EP 6058 Csteps= 52775 - 8 steps - CReward: 9.38  Eps=0.1  Time: 3:10:38
EP 6059 Csteps= 52790 - 15 steps - CReward: 11.37  Eps=0.1  Time: 3:10:41
EP 6060 Csteps= 52799 - 9 steps - CReward: -2.29  Eps=0.1  Time: 3:10:43
EP 6061 Csteps= 52805 - 6 steps - CReward: 11.15  Eps=0.1  Time: 3:10:44
EP 6062 Csteps= 52821 - 16 steps - CReward: 10.45  Eps=0.1  Time: 3:10:48
EP 6063 Csteps= 52836 - 15 steps - CReward: 9.02  Eps=0.1  Time: 3:10:51
EP 6064 Csteps= 52845 - 9 steps - CReward: 11.46  Eps=0.1  Time: 3:10:53
EP 6065 Csteps= 52858 - 13 steps - CReward: 10.29  Eps=0.1  Time: 3:10:56
EP 6066 Csteps= 52868 - 10 steps - CReward: 0.35  Eps=0.1  Time: 3:10:59
EP 6067 Csteps= 52887 - 19 steps - CReward: 10.69  Eps=0.1  Time: 3:11:03
EP 6068 Csteps= 52892 - 5 steps - CReward: 3.89  E

EP 6168 Csteps= 53836 - 12 steps - CReward: 9.57  Eps=0.1  Time: 3:14:49
EP 6169 Csteps= 53848 - 12 steps - CReward: 8.3  Eps=0.1  Time: 3:14:52
EP 6170 Csteps= 53854 - 6 steps - CReward: 11.26  Eps=0.1  Time: 3:14:54
EP 6171 Csteps= 53873 - 19 steps - CReward: 11.16  Eps=0.1  Time: 3:14:58
EP 6172 Csteps= 53880 - 7 steps - CReward: 10.17  Eps=0.1  Time: 3:15:00
EP 6173 Csteps= 53894 - 14 steps - CReward: 10.53  Eps=0.1  Time: 3:15:03
EP 6174 Csteps= 53912 - 18 steps - CReward: 8.42  Eps=0.1  Time: 3:15:07
EP 6175 Csteps= 53920 - 8 steps - CReward: 11.26  Eps=0.1  Time: 3:15:08
EP 6176 Csteps= 53933 - 13 steps - CReward: -0.05  Eps=0.1  Time: 3:15:11
EP 6177 Csteps= 53940 - 7 steps - CReward: 10.61  Eps=0.1  Time: 3:15:13
EP 6178 Csteps= 53948 - 8 steps - CReward: 11.31  Eps=0.1  Time: 3:15:15
EP 6179 Csteps= 53960 - 12 steps - CReward: 9.67  Eps=0.1  Time: 3:15:18
EP 6180 Csteps= 53969 - 9 steps - CReward: 9.63  Eps=0.1  Time: 3:15:20
EP 6181 Csteps= 53977 - 8 steps - CReward: 10.02  

In [ ]:
import torch
from thop import profile, clever_format


input = (torch.randn(1, 3, 84, 84)).cuda()
macs, params = profile(Agent.model, inputs=(input, ))
macs, params = clever_format([macs, params], "%.3f")

print('Parameters: {}\t  MACs: {}'.format(params, macs))